In [2]:
import nest_asyncio
import pandas as pd
import numpy as np
from tastytrade import Account, Session, DXLinkStreamer
from tastytrade.dxfeed import Quote, Candle
from tastytrade.instruments import *
from tastytrade.market_data import *
import yfinance as yf
from itertools import chain
from config import *
from alphas import *
from auxiliary import *
from datetime import datetime, timedelta, time
import asyncio
import requests
import logging
import json
nest_asyncio.apply()

session=None
account=None

async def setup():
    global session, account, config
    # setup
    config = Config(test=False)
    session = Session(config.username, config.password, is_test=config.test)
    account = await Account.a_get(session, config.account_number)

    # account info
    balance = account.get_balances(session)
    positions = account.get_positions(session)
    history = account.get_history(session)

asyncio.run(setup())
print("Session and account setup complete.")

streamer = DXLinkStreamer(session) #async streamer

Session and account setup complete.


In [7]:
SPXoptionchain = get_option_chain(session, "SPX")
VIXoptionchain = get_option_chain(session, "VIX")

SPXexpiries = list(SPXoptionchain.keys())
VIXexpiries = list(VIXoptionchain.keys())

SPXdata = get_market_data_by_type(session,indices = ['SPX'])
VIXdata = get_market_data_by_type(session,indices = ['VIX'])
print(SPXdata)

[MarketData(symbol='SPX' instrument_type=<InstrumentType.INDEX: 'Index'> updated_at=datetime.datetime(2025, 6, 12, 5, 36, 20, 909000, tzinfo=TzInfo(UTC)) mark=Decimal('6017.51') close_price_type=<ClosePriceType.REGULAR: 'Regular'> summary_date=datetime.date(2025, 6, 12) prev_close_date=datetime.date(2025, 6, 11) prev_close_price_type=<ClosePriceType.REGULAR: 'Regular'> halt_start_time=-1 halt_end_time=-1 ask=Decimal('6067.86') beta=Decimal('1.0') bid=Decimal('5967.16') day_high_price=Decimal('6059.4') day_low_price=Decimal('6002.32') last=Decimal('6022.24') last_mkt=Decimal('6022.24') mid=Decimal('6017.51') open=Decimal('6049.38') prev_close=Decimal('6022.24') year_low_price=Decimal('4835.04') year_high_price=Decimal('6147.43'))]


In [5]:
date.today()

datetime.date(2025, 6, 12)

In [6]:
first5=SPXexpiries[:5]
sd=first5[-1]+timedelta(days=1)
def ldm(d): nd=datetime(d.year+1,1,1) if d.month==12 else datetime(d.year,d.month+1,1); return nd-timedelta(days=1)
eoms=[d for d in SPXexpiries if d>=sd and d.day==ldm(d).day][:6]
SPXsavelist=first5+eoms
VIXsavelist=VIXexpiries[:10]

In [10]:
SPXspot = SPXdata[0].last
VIXspot = VIXdata[0].last

In [14]:
payload = {
    "SPX": {"spot": SPXspot},
    "VIX": {"spot": VIXspot}
}

for exp in SPXsavelist:
    payload["SPX"][exp.isoformat()] = {
        "optionchain": OptionMethods.convertchain(session, SPXoptionchain[exp])
    }

for exp in VIXsavelist:
    payload["VIX"][exp.isoformat()] = {
        "optionchain": OptionMethods.convertchain(session, VIXoptionchain[exp])
    }

snapshot = {
    "date": date.today().isoformat(),
    "period": "AM" if datetime.now().time() < time(12, 0) else "PM",
    "fetchtime": datetime.now().isoformat(),
    "rfr": 4.5,
    "payload": payload
}